In [49]:
import pandas as pd
import random
import GEOparse
import time
import requests
import os

from urllib3.exceptions import IncompleteRead

In [50]:
GEOparse.downloader.DOWNLOAD_METHOD = "https"

In [51]:
datasets_df = pd.read_excel('Magalhaes_datasets.xlsx', header=1)


In [52]:
datasets_df.head()

,Study,Tissue,Species,Technology
0,AgeMap,Adrenal Glands,Mouse,Microarray
1,AgeMap,Bone Marrow,Mouse,Microarray
2,AgeMap,Brain,Mouse,Microarray
3,AgeMap,Eye,Mouse,Microarray
4,GDS1079,Myoblasts,Mouse,Microarray


In [53]:
gse_id_list = datasets_df.query("Technology=='Microarray' & Study !='AgeMap'").Study.tolist()


In [54]:
gse_id_list

['GDS1079',
 'GDS1278',
 'GDS1279',
 'GDS1280',
 'GDS1280',
 'GDS1311',
 'GDS156',
 'GDS1647',
 'GDS1803',
 'GDS2019',
 'GDS2082',
 'GDS2612',
 'GDS2639',
 'GDS2681',
 'GDS287',
 'GDS288',
 'GDS2912',
 'GDS2929',
 'GDS2972',
 'GDS2973',
 'GDS2996',
 'GDS3077',
 'GDS3102',
 'GDS3102',
 'GDS3182',
 'GDS355',
 'GDS356',
 'GDS3620',
 'GDS3869',
 'GDS3915',
 'GDS3939',
 'GDS3976',
 'GDS399',
 'GDS40',
 'GDS4019',
 'GDS4264',
 'GDS4522',
 'GDS4523',
 'GDS472',
 'GDS473',
 'GDS4804',
 'GDS4858',
 'GDS4874',
 'GDS4892',
 'GDS4904',
 'GDS4925',
 'GDS520',
 'GDS5204',
 'GDS5216',
 'GDS5217',
 'GDS5218',
 'GDS5226',
 'GDS5286',
 'GDS5412',
 'GDS707',
 'GSE11097',
 'GSE11291',
 'GSE11291',
 'GSE11291',
 'GSE12277',
 'GSE12480',
 'GSE16487',
 'GSE19676',
 'GSE21681',
 'GSE27625',
 'GSE32719',
 'GSE34378',
 'GSE34378',
 'GSE34378',
 'GSE34378',
 'GSE34378',
 'GSE35322',
 'GSE36192',
 'GSE39540',
 'GSE40645',
 'GSE48911',
 'GSE5086',
 'GSE56045',
 'GSE56580',
 'GSE58137',
 'GSE61915',
 'GSE63060',
 '

In [55]:
gse_id_list = [gse_id for gse_id in gse_id_list if gse_id.startswith('GSE')]


In [56]:
gse_id_list

['GSE11097',
 'GSE11291',
 'GSE11291',
 'GSE11291',
 'GSE12277',
 'GSE12480',
 'GSE16487',
 'GSE19676',
 'GSE21681',
 'GSE27625',
 'GSE32719',
 'GSE34378',
 'GSE34378',
 'GSE34378',
 'GSE34378',
 'GSE34378',
 'GSE35322',
 'GSE36192',
 'GSE39540',
 'GSE40645',
 'GSE48911',
 'GSE5086',
 'GSE56045',
 'GSE56580',
 'GSE58137',
 'GSE61915',
 'GSE63060',
 'GSE66857',
 'GSE69187',
 'GSE69832',
 'GSE73450',
 'GSE74463',
 'GSE7958',
 'GSE85084']

In [57]:
# has_relationships = []

# for gse_id in gse_id_list:
    
#         gse = GEOparse.get_GEO(geo=gse_id, destdir="./data/")
        
#         gsm_ids = list(gse.gsms.keys())
        
#         for gsm_id in gsm_ids:
#             has_relationships.append(f"has({gse_id}, {gsm_id});")    

In [59]:
def download_file(url, dest_path, timeout=60):
    try:
        response = requests.get(url, stream=True, timeout=timeout)
        response.raise_for_status()  
        
        with open(dest_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
        return True
    except IncompleteRead as e:
        print(f"Incomplete read error: {e}")
        return False
    except requests.exceptions.RequestException as e:
        print(f"Failed to download {url}: {e}")
        return False

def download_gse_with_retries(gse_id, max_retries=5):
    retries = 0
    while retries < max_retries:
        try:
            url = f"https://ftp.ncbi.nlm.nih.gov/geo/series/{gse_id[:-3]}nnn/{gse_id}/soft/{gse_id}_family.soft.gz"
            dest_path = f"./data/{gse_id}_family.soft.gz"
            
            print(f"Downloading {gse_id} (Attempt {retries + 1}/{max_retries})...")
            
            if download_file(url, dest_path):
                gse = GEOparse.get_GEO(filepath=dest_path)
                return gse
        except Exception as e:
            print(f"Attempt {retries + 1} failed for {gse_id}: {e}")
        
        retries += 1
        time.sleep(10)  

    print(f"Failed to download {gse_id} after {max_retries} attempts.")
    return None


In [60]:
has_relationships = []

for gse_id in gse_id_list:
    gse = download_gse_with_retries(gse_id)
    if gse:  
        gsm_ids = list(gse.gsms.keys())
        for gsm_id in gsm_ids:
            has_relationships.append(f"has({gse_id}, {gsm_id});") 


22-Mar-2025 22:14:12 INFO GEOparse - Parsing ./data/GSE11097_family.soft.gz: 
22-Mar-2025 22:14:12 DEBUG GEOparse - DATABASE: GeoMiame
22-Mar-2025 22:14:12 DEBUG GEOparse - SERIES: GSE11097
22-Mar-2025 22:14:12 DEBUG GEOparse - PLATFORM: GPL1355
22-Mar-2025 22:14:14 DEBUG GEOparse - SAMPLE: GSM280267
22-Mar-2025 22:14:14 DEBUG GEOparse - SAMPLE: GSM280268
22-Mar-2025 22:14:14 DEBUG GEOparse - SAMPLE: GSM280269
22-Mar-2025 22:14:14 DEBUG GEOparse - SAMPLE: GSM280270
22-Mar-2025 22:14:14 DEBUG GEOparse - SAMPLE: GSM280271
22-Mar-2025 22:14:14 DEBUG GEOparse - SAMPLE: GSM280272
22-Mar-2025 22:14:14 DEBUG GEOparse - SAMPLE: GSM280273
22-Mar-2025 22:14:14 DEBUG GEOparse - SAMPLE: GSM280274
22-Mar-2025 22:14:15 DEBUG GEOparse - SAMPLE: GSM280275
22-Mar-2025 22:14:15 DEBUG GEOparse - SAMPLE: GSM280276
22-Mar-2025 22:14:15 DEBUG GEOparse - SAMPLE: GSM280277
22-Mar-2025 22:14:15 DEBUG GEOparse - SAMPLE: GSM280278
22-Mar-2025 22:14:15 DEBUG GEOparse - SAMPLE: GSM280279
22-Mar-2025 22:14:15 DEBUG

22-Mar-2025 22:15:45 INFO GEOparse - Parsing ./data/GSE11291_family.soft.gz: 
22-Mar-2025 22:15:45 DEBUG GEOparse - DATABASE: GeoMiame
22-Mar-2025 22:15:45 DEBUG GEOparse - SERIES: GSE11291
22-Mar-2025 22:15:45 DEBUG GEOparse - PLATFORM: GPL1261
c:\USERS\HIWOT\DESKTOP\ICOG\REPRESENTS THE RELATIONSHIP\MYENV\Lib\site-packages\GEOparse\GEOparse.py:401: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  return read_csv(StringIO(data), index_col=None, sep="\t")
22-Mar-2025 22:15:47 DEBUG GEOparse - SAMPLE: GSM284967
22-Mar-2025 22:15:48 DEBUG GEOparse - SAMPLE: GSM284968
22-Mar-2025 22:15:48 DEBUG GEOparse - SAMPLE: GSM284969
22-Mar-2025 22:15:48 DEBUG GEOparse - SAMPLE: GSM284970
22-Mar-2025 22:15:48 DEBUG GEOparse - SAMPLE: GSM284971
22-Mar-2025 22:15:48 DEBUG GEOparse - SAMPLE: GSM284972
22-Mar-2025 22:15:48 DEBUG GEOparse - SAMPLE: GSM284973
22-Mar-2025 22:15:49 DEBUG GEOparse - SAMPLE: GSM284974
22-Mar-2025 22:15:49 DEBUG GEOparse - SA

22-Mar-2025 22:17:51 INFO GEOparse - Parsing ./data/GSE11291_family.soft.gz: 
22-Mar-2025 22:17:51 DEBUG GEOparse - DATABASE: GeoMiame
22-Mar-2025 22:17:51 DEBUG GEOparse - SERIES: GSE11291
22-Mar-2025 22:17:51 DEBUG GEOparse - PLATFORM: GPL1261
c:\USERS\HIWOT\DESKTOP\ICOG\REPRESENTS THE RELATIONSHIP\MYENV\Lib\site-packages\GEOparse\GEOparse.py:401: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  return read_csv(StringIO(data), index_col=None, sep="\t")
22-Mar-2025 22:17:53 DEBUG GEOparse - SAMPLE: GSM284967
22-Mar-2025 22:17:53 DEBUG GEOparse - SAMPLE: GSM284968
22-Mar-2025 22:17:53 DEBUG GEOparse - SAMPLE: GSM284969
22-Mar-2025 22:17:53 DEBUG GEOparse - SAMPLE: GSM284970
22-Mar-2025 22:17:53 DEBUG GEOparse - SAMPLE: GSM284971
22-Mar-2025 22:17:54 DEBUG GEOparse - SAMPLE: GSM284972
22-Mar-2025 22:17:54 DEBUG GEOparse - SAMPLE: GSM284973
22-Mar-2025 22:17:54 DEBUG GEOparse - SAMPLE: GSM284974
22-Mar-2025 22:17:54 DEBUG GEOparse - SA

22-Mar-2025 22:19:47 INFO GEOparse - Parsing ./data/GSE11291_family.soft.gz: 
22-Mar-2025 22:19:47 DEBUG GEOparse - DATABASE: GeoMiame
22-Mar-2025 22:19:47 DEBUG GEOparse - SERIES: GSE11291
22-Mar-2025 22:19:47 DEBUG GEOparse - PLATFORM: GPL1261
c:\USERS\HIWOT\DESKTOP\ICOG\REPRESENTS THE RELATIONSHIP\MYENV\Lib\site-packages\GEOparse\GEOparse.py:401: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  return read_csv(StringIO(data), index_col=None, sep="\t")
22-Mar-2025 22:19:49 DEBUG GEOparse - SAMPLE: GSM284967
22-Mar-2025 22:19:49 DEBUG GEOparse - SAMPLE: GSM284968
22-Mar-2025 22:19:50 DEBUG GEOparse - SAMPLE: GSM284969
22-Mar-2025 22:19:50 DEBUG GEOparse - SAMPLE: GSM284970
22-Mar-2025 22:19:50 DEBUG GEOparse - SAMPLE: GSM284971
22-Mar-2025 22:19:50 DEBUG GEOparse - SAMPLE: GSM284972
22-Mar-2025 22:19:50 DEBUG GEOparse - SAMPLE: GSM284973
22-Mar-2025 22:19:50 DEBUG GEOparse - SAMPLE: GSM284974
22-Mar-2025 22:19:51 DEBUG GEOparse - SA

22-Mar-2025 22:21:05 INFO GEOparse - Parsing ./data/GSE12277_family.soft.gz: 
22-Mar-2025 22:21:05 DEBUG GEOparse - DATABASE: GeoMiame
22-Mar-2025 22:21:05 DEBUG GEOparse - SERIES: GSE12277
22-Mar-2025 22:21:05 DEBUG GEOparse - PLATFORM: GPL570
c:\USERS\HIWOT\DESKTOP\ICOG\REPRESENTS THE RELATIONSHIP\MYENV\Lib\site-packages\GEOparse\GEOparse.py:401: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  return read_csv(StringIO(data), index_col=None, sep="\t")
22-Mar-2025 22:21:07 DEBUG GEOparse - SAMPLE: GSM308237
22-Mar-2025 22:21:07 DEBUG GEOparse - SAMPLE: GSM308239
22-Mar-2025 22:21:07 DEBUG GEOparse - SAMPLE: GSM308241
22-Mar-2025 22:21:07 DEBUG GEOparse - SAMPLE: GSM308242
22-Mar-2025 22:21:08 DEBUG GEOparse - SAMPLE: GSM308244
22-Mar-2025 22:21:08 DEBUG GEOparse - SAMPLE: GSM308245
22-Mar-2025 22:21:08 DEBUG GEOparse - SAMPLE: GSM308246
22-Mar-2025 22:21:08 DEBUG GEOparse - SAMPLE: GSM308247
22-Mar-2025 22:21:08 DEBUG GEOparse - SAM

...

23-Mar-2025 00:10:34 INFO GEOparse - Parsing ./data/GSE69832_family.soft.gz: 
23-Mar-2025 00:10:34 DEBUG GEOparse - DATABASE: GeoMiame
23-Mar-2025 00:10:34 DEBUG GEOparse - SERIES: GSE69832
23-Mar-2025 00:10:34 DEBUG GEOparse - PLATFORM: GPL6244
23-Mar-2025 00:10:37 DEBUG GEOparse - SAMPLE: GSM1709748
23-Mar-2025 00:10:37 DEBUG GEOparse - SAMPLE: GSM1709749
23-Mar-2025 00:10:37 DEBUG GEOparse - SAMPLE: GSM1709750
23-Mar-2025 00:10:37 DEBUG GEOparse - SAMPLE: GSM1709751
23-Mar-2025 00:10:37 DEBUG GEOparse - SAMPLE: GSM1709752
23-Mar-2025 00:10:37 DEBUG GEOparse - SAMPLE: GSM1709753
23-Mar-2025 00:10:37 DEBUG GEOparse - SAMPLE: GSM1709754
23-Mar-2025 00:10:37 DEBUG GEOparse - SAMPLE: GSM1709755
23-Mar-2025 00:10:37 DEBUG GEOparse - SAMPLE: GSM1709756
23-Mar-2025 00:10:37 DEBUG GEOparse - SAMPLE: GSM1709757
23-Mar-2025 00:10:37 DEBUG GEOparse - SAMPLE: GSM1709758
23-Mar-2025 00:10:37 DEBUG GEOparse - SAMPLE: GSM1709759
23-Mar-2025 00:10:37 DEBUG GEOparse - SAMPLE: GSM1709760
23-Mar-2025 0

23-Mar-2025 00:12:09 INFO GEOparse - Parsing ./data/GSE73450_family.soft.gz: 
23-Mar-2025 00:12:09 DEBUG GEOparse - DATABASE: GeoMiame
23-Mar-2025 00:12:09 DEBUG GEOparse - SERIES: GSE73450
23-Mar-2025 00:12:09 DEBUG GEOparse - PLATFORM: GPL6887
23-Mar-2025 00:12:12 DEBUG GEOparse - SAMPLE: GSM1894064
23-Mar-2025 00:12:12 DEBUG GEOparse - SAMPLE: GSM1894065
23-Mar-2025 00:12:12 DEBUG GEOparse - SAMPLE: GSM1894066
23-Mar-2025 00:12:12 DEBUG GEOparse - SAMPLE: GSM1894067
23-Mar-2025 00:12:12 DEBUG GEOparse - SAMPLE: GSM1894068
23-Mar-2025 00:12:12 DEBUG GEOparse - SAMPLE: GSM1894069
23-Mar-2025 00:12:13 DEBUG GEOparse - SAMPLE: GSM1894070
23-Mar-2025 00:12:13 DEBUG GEOparse - SAMPLE: GSM1894071
23-Mar-2025 00:12:13 DEBUG GEOparse - SAMPLE: GSM1894072
23-Mar-2025 00:12:13 DEBUG GEOparse - SAMPLE: GSM1894073
23-Mar-2025 00:12:13 DEBUG GEOparse - SAMPLE: GSM1894074
23-Mar-2025 00:12:13 DEBUG GEOparse - SAMPLE: GSM1894075
23-Mar-2025 00:12:14 DEBUG GEOparse - SAMPLE: GSM1894076
23-Mar-2025 0

23-Mar-2025 00:12:45 INFO GEOparse - Parsing ./data/GSE74463_family.soft.gz: 
23-Mar-2025 00:12:45 DEBUG GEOparse - DATABASE: GeoMiame
23-Mar-2025 00:12:45 DEBUG GEOparse - SERIES: GSE74463
23-Mar-2025 00:12:45 DEBUG GEOparse - PLATFORM: GPL6246
23-Mar-2025 00:12:47 DEBUG GEOparse - SAMPLE: GSM1921098
23-Mar-2025 00:12:47 DEBUG GEOparse - SAMPLE: GSM1921099
23-Mar-2025 00:12:48 DEBUG GEOparse - SAMPLE: GSM1921100
23-Mar-2025 00:12:48 DEBUG GEOparse - SAMPLE: GSM1921101
23-Mar-2025 00:12:48 DEBUG GEOparse - SAMPLE: GSM1921102
23-Mar-2025 00:12:48 DEBUG GEOparse - SAMPLE: GSM1921103
23-Mar-2025 00:12:48 DEBUG GEOparse - SAMPLE: GSM1921104
23-Mar-2025 00:12:48 DEBUG GEOparse - SAMPLE: GSM1921105
23-Mar-2025 00:12:48 DEBUG GEOparse - SAMPLE: GSM1921106


23-Mar-2025 00:13:10 INFO GEOparse - Parsing ./data/GSE7958_family.soft.gz: 
23-Mar-2025 00:13:10 DEBUG GEOparse - DATABASE: GeoMiame
23-Mar-2025 00:13:10 DEBUG GEOparse - SERIES: GSE7958
23-Mar-2025 00:13:10 DEBUG GEOparse - PLATFORM: GPL1261
c:\USERS\HIWOT\DESKTOP\ICOG\REPRESENTS THE RELATIONSHIP\MYENV\Lib\site-packages\GEOparse\GEOparse.py:401: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  return read_csv(StringIO(data), index_col=None, sep="\t")
23-Mar-2025 00:13:12 DEBUG GEOparse - SAMPLE: GSM197114
23-Mar-2025 00:13:13 DEBUG GEOparse - SAMPLE: GSM197115
23-Mar-2025 00:13:13 DEBUG GEOparse - SAMPLE: GSM197116
23-Mar-2025 00:13:13 DEBUG GEOparse - SAMPLE: GSM197117
23-Mar-2025 00:13:13 DEBUG GEOparse - SAMPLE: GSM197118
23-Mar-2025 00:13:13 DEBUG GEOparse - SAMPLE: GSM197119
23-Mar-2025 00:13:13 DEBUG GEOparse - SAMPLE: GSM197120
23-Mar-2025 00:13:13 DEBUG GEOparse - SAMPLE: GSM197121
23-Mar-2025 00:13:14 DEBUG GEOparse - SAMP

23-Mar-2025 00:13:35 INFO GEOparse - Parsing ./data/GSE85084_family.soft.gz: 
23-Mar-2025 00:13:35 DEBUG GEOparse - DATABASE: GeoMiame
23-Mar-2025 00:13:35 DEBUG GEOparse - SERIES: GSE85084
23-Mar-2025 00:13:35 DEBUG GEOparse - PLATFORM: GPL6885
23-Mar-2025 00:13:36 DEBUG GEOparse - SAMPLE: GSM2257446
23-Mar-2025 00:13:36 DEBUG GEOparse - SAMPLE: GSM2257447
23-Mar-2025 00:13:36 DEBUG GEOparse - SAMPLE: GSM2257448
23-Mar-2025 00:13:36 DEBUG GEOparse - SAMPLE: GSM2257449
23-Mar-2025 00:13:36 DEBUG GEOparse - SAMPLE: GSM2257450
23-Mar-2025 00:13:36 DEBUG GEOparse - SAMPLE: GSM2257451
23-Mar-2025 00:13:37 DEBUG GEOparse - SAMPLE: GSM2257452
23-Mar-2025 00:13:37 DEBUG GEOparse - SAMPLE: GSM2257453
23-Mar-2025 00:13:37 DEBUG GEOparse - SAMPLE: GSM2257454
23-Mar-2025 00:13:37 DEBUG GEOparse - SAMPLE: GSM2257455
23-Mar-2025 00:13:37 DEBUG GEOparse - SAMPLE: GSM2257456
23-Mar-2025 00:13:37 DEBUG GEOparse - SAMPLE: GSM2257457
23-Mar-2025 00:13:37 DEBUG GEOparse - SAMPLE: GSM2257458
23-Mar-2025 0

In [61]:
for relationship in has_relationships:
    print(relationship)

has(GSE11097, GSM280267);
has(GSE11097, GSM280268);
has(GSE11097, GSM280269);
has(GSE11097, GSM280270);
has(GSE11097, GSM280271);
has(GSE11097, GSM280272);
has(GSE11097, GSM280273);
has(GSE11097, GSM280274);
has(GSE11097, GSM280275);
has(GSE11097, GSM280276);
has(GSE11097, GSM280277);
has(GSE11097, GSM280278);
has(GSE11097, GSM280279);
has(GSE11097, GSM280280);
has(GSE11097, GSM280281);
has(GSE11097, GSM280282);
has(GSE11097, GSM280283);
has(GSE11097, GSM280284);
has(GSE11097, GSM280394);
has(GSE11097, GSM280395);
has(GSE11097, GSM280396);
has(GSE11097, GSM280397);
has(GSE11097, GSM280398);
has(GSE11097, GSM280399);
has(GSE11097, GSM280400);
has(GSE11097, GSM280401);
has(GSE11097, GSM280402);
has(GSE11097, GSM280403);
has(GSE11097, GSM280404);
has(GSE11097, GSM280405);
has(GSE11097, GSM280406);
has(GSE11097, GSM280407);
has(GSE11097, GSM280408);
has(GSE11097, GSM280409);
has(GSE11291, GSM284967);
has(GSE11291, GSM284968);
has(GSE11291, GSM284969);
has(GSE11291, GSM284970);
has(GSE11291

In [62]:
with open("gse_gsm_relationships.txt", "w") as f:
    for relationship in has_relationships:
        f.write(relationship + "\n")